In [1]:
# reference

#https://www.kaggle.com/code/ayuraj/train-covid-19-detection-using-yolov5/notebook


In [2]:
import os

 
``` /parent_folder
    /dataset
         /images
         /labels
    /yolov5
```
のような階層構造にする


In [3]:
%cd ../
!mkdir tmp

/kaggle


In [4]:
%ls

input/  lib/  src/  tmp/  working/


In [5]:
#!cp -r "input/rdd-japan/data" "tmp/"

In [6]:
"""os.makedirs('/tmp/rdd/images/train', exist_ok=True)
os.makedirs('/tmp/rdd/images/valid', exist_ok=True)

os.makedirs('/tmp/rdd/labels/train', exist_ok=True)
os.makedirs('/tmp/rdd/labels/valid', exist_ok=True)"""


"os.makedirs('/tmp/rdd/images/train', exist_ok=True)\nos.makedirs('/tmp/rdd/images/valid', exist_ok=True)\n\nos.makedirs('/tmp/rdd/labels/train', exist_ok=True)\nos.makedirs('/tmp/rdd/labels/valid', exist_ok=True)"

In [7]:
%ls tmp/rdd

ls: cannot access 'tmp/rdd': No such file or directory


In [8]:
%ls input

rdd-japan/  yolov5-lib-ds/


In [9]:
#%cp "input/rdd-japan/data/Train/images/*" "tmp/rdd/images/train/"

In [10]:
#%cp "input/rdd-japan/data/Train/labels/" 'tmp/rdd/labels/train'

In [11]:
#%cp -r "input/rdd-japan/data/Valid/labels/" 'tmp/rdd/labels/valid'

In [12]:
#%cp -r "input/rdd-japan/data/Valid/images/" 'tmp/rdd/images/valid'

In [13]:
#%ls 'tmp/rdd/images/valid/'

In [14]:
%cd tmp

/kaggle/tmp


In [15]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


Cloning into 'yolov5'...
remote: Enumerating objects: 11588, done.
remote: Total 11588 (delta 0), reused 0 (delta 0), pack-reused 11588
Receiving objects: 100% (11588/11588), 11.33 MiB | 31.03 MiB/s, done.
Resolving deltas: 100% (8011/8011), done.
/kaggle/tmp/yolov5
Note: you may need to restart the kernel to use updated packages.
/kaggle/tmp
Setup complete. Using torch 1.9.1 (Tesla P100-PCIE-16GB)


In [16]:
os.makedirs('../tmp/yolov5/data/images/train', exist_ok=True)
os.makedirs('../tmp/yolov5/data/images/valid', exist_ok=True)

os.makedirs('../tmp/yolov5/data/labels/train', exist_ok=True)
os.makedirs('../tmp/yolov5/data/labels/valid', exist_ok=True)

In [17]:
#%cp "../../kaggle/input/rdd-japan/data/Train/images/" "../tmp/yolov5/data/images/train/"

In [18]:
%cp -r "../../kaggle/input/rdd-japan/data/Train/" "../tmp/yolov5/data/"

In [19]:
%cp -r "../../kaggle/input/rdd-japan/data/Valid/" "../tmp/yolov5/data/"

In [20]:
%ls "../tmp/yolov5/data/"

Argoverse.yaml        SKU-110K.yaml  Valid/         coco128.yaml  labels/
GlobalWheat2020.yaml  Train/         VisDrone.yaml  hyps/         scripts/
Objects365.yaml       VOC.yaml       coco.yaml      images/       xView.yaml


In [21]:
%ls "../tmp/yolov5/data/Train"

images/  labels/


In [22]:
# Install W&B 
!pip install -q --upgrade wandb
# Login 
import wandb
#wandb.login()

In [23]:
# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [24]:
# Everything is done from /kaggle directory.
%cd ../

/kaggle


In [25]:
import yaml

classes_kaggle = {
    
"train": "data/Train",
"val": "data/Valid",
#"test": "../Road_Sign_Dataset/images/test/",

# number of classes
"nc": 9,

# class names
"names": ["D00","D01","D10","D11","D20",
          "D40","D43","D44","D50"]
}

with open('tmp/yolov5/data/data.yaml', 'w') as file:
    yaml.dump(classes_kaggle, file)

%cat tmp/yolov5/data/data.yaml

names:
- D00
- D01
- D10
- D11
- D20
- D40
- D43
- D44
- D50
nc: 9
train: data/Train
val: data/Valid


In [26]:
DIM       = 3000 
MODEL     = 'yolov5s6'
BATCH     = 4
IMG_SIZE = 600
BATCH_SIZE = 16
EPOCHS = 2
OPTMIZER  = 'Adam'

PROJECT   = 'RDD' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}_Epoch{EPOCHS}' # w&b for yolov5

In [27]:
%cd tmp/yolov5

/kaggle/tmp/yolov5


In [28]:
%ls ../../tmp

yolov5/


In [29]:
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data ./data/data.yaml \
                 --weights yolov5s.pt \
                 --project rdd

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
100%|███████████████████████████████████████| 14.1M/14.1M [00:00<00:00, 127MB/s]

train: Scanning '/kaggle/tmp/yolov5/data/Train/labels' images and labels...7354 
val: Scanning '/kaggle/tmp/yolov5/data/Valid/labels' images and labels...3152 fo
       0/1     3.15G   0.08433   0.03189   0.04774        28       608: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
       1/1     3.57G    0.0638   0.02815   0.02714        36       608: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa23797fa70>
Traceback (most recent call last):
  File "/opt/conda/lib/pytho

In [30]:
%pwd

'/kaggle/tmp/yolov5'

In [31]:
%ls ../../

input/  lib/  src/  tmp/  working/


In [32]:
%cp -r ../../tmp/yolov5/ /kaggle/working/

In [33]:
"""!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data ./data/data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project rdd"""

'!python train.py --img {IMG_SIZE}                  --batch {BATCH_SIZE}                  --epochs {EPOCHS}                  --data ./data/data.yaml                  --weights yolov5s.pt                  --save_period 1                 --project rdd'

In [34]:
%ls ../../tmp

yolov5/
